## Spark Load Serverlogs to Hive

In [11]:
# Módulos usados
import sys  
import pandas as pd
import numpy as np
from pyspark import SparkContext
import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext


In [2]:
sc = SparkContext()
spark = SparkSession(sc)
hqlContext = HiveContext(sc)

In [3]:
# Lendo o arquivo texto e criando um RDD em memória com Spark
line = sc.textFile("E:/Google Drive/Meu Disco/work/Treinamentos/DSA/Engenharia de dados com Hadoop e Spark/Projetos/Projeto6/serverlogs/access_log_Jul95_amostra")

In [4]:
# Essa função separa as colunas em cada linha, cria uma tupla e remove a pontuação.
def get_row(row):
    host = row[0:row.find(' - -')]
    timestamp = row[row.find('[')+1:row.find(']')]
    request = row[row.find('"')+1:row.rfind('"')]
    http = row[row.rfind('"')+2:row.rfind('"')+5]
    bytes = row[row.rfind('"')+6:]
    
    return {'host':host, 'timestamp':timestamp, 'request':request, 'http':http, 'bytes':bytes}

In [5]:
# Aplica a função a cada linha do arquivo
dataset = line.map(lambda line: get_row(line))

In [12]:
serverlogs_df = dataset.toDF()


AnalysisException: u'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

In [9]:
#serverlogs_df.show()
dataset.to

AttributeError: 'PipelinedRDD' object has no attribute 'show'

In [9]:
serverlogs_pd = serverlogs_df.toPandas()

In [10]:
serverlogs_pd['bytes'] = pd.to_numeric(serverlogs_pd['bytes'], errors='coerce')
serverlogs_pd['data'] = pd.to_datetime(serverlogs_pd.timestamp.str[:11])

In [12]:

serverlogs_pd.head()

,bytes,host,http,request,timestamp,data
0,6245,199.72.81.55,200,GET /history/apollo/ HTTP/1.0,01/Jul/1995:00:00:01 -0400,1995-07-01
1,3985,unicomp6.unicomp.net,200,GET /shuttle/countdown/ HTTP/1.0,01/Jul/1995:00:00:06 -0400,1995-07-01
2,4085,199.120.110.21,200,GET /shuttle/missions/sts-73/mission-sts-73.ht...,01/Jul/1995:00:00:09 -0400,1995-07-01
3,0,burger.letters.com,304,GET /shuttle/countdown/liftoff.html HTTP/1.0,01/Jul/1995:00:00:11 -0400,1995-07-01
4,4179,199.120.110.21,200,GET /shuttle/missions/sts-73/sts-73-patch-smal...,01/Jul/1995:00:00:11 -0400,1995-07-01


In [16]:

serverlogs_hdf = hqlContext.createDataFrame(serverlogs_pd)

In [17]:
serverlogs_hdf.registerTempTable('serverlogs_tmp')

In [26]:
sql_cmd = '''
SET hive.exec.dynamic.partition = true;
SET hive.exec.dynamic.partition.mode = nonstrict;

INSERT INTO TABLE default.t_serverlogs
PARTITION (data)
SELECT 
    bytes,
    host,
    http,
    request,
    data
FROM serverlogs_tmp;
'''

In [27]:
hqlContext.sql(sql_command)

DataFrame[key: string, value: string]


SET hive.exec.dynamic.partition = true;
SET hive.exec.dynamic.partition.mode = nonstrict;

INSERT INTO TABLE default.t_serverlogs
PARTITION (data)
SELECT 
    bytes,
    host,
    http,
    request,
    data
FROM serverlogs_tmp;



# Fim

### Obrigado - Data Science Academy - <a href="http://facebook.com/dsacademybr">facebook.com/dsacademybr</a>